This notebook is based on: https://esciencecenter-digital-skills.github.io/geospatial-python/06-raster-intro.html

In [ ]:
import rioxarray


Explain relationship rioxarray xarray rastterio pyproj and numpy

In [ ]:
!pip show rioxarray

In [ ]:
rioxarray.open_rasterio?

In [ ]:
red = rioxarray.open_rasterio('../data/input_data/sentinel2/red.tif')

In [ ]:
red

In [ ]:
red.values

In [ ]:
print(red.max())
print(red.min())

In [ ]:
print(red.rio.crs)

In [ ]:
red.plot()

In [ ]:
red_lowres = rioxarray.open_rasterio('../data/input_data/sentinel2/red.tif', overview_level=2)

In [ ]:
red_lowres

In [ ]:
red_lowres.plot()

In [ ]:
red_lowres.plot(robust=True)

## View Raster Coordinate Reference System (CRS) in Python

In [ ]:
red.rio.crs

In [ ]:
epsg = red.rio.crs.to_epsg()
epsg 

In [ ]:
from pyproj import CRS
crs = CRS(epsg)
crs

## Dealing with Missing Data

In [ ]:
red_nodata = rioxarray.open_rasterio('../data/input_data/sentinel2/red.tif', masked=True, overview_level=2)

In [ ]:
red_nodata

In [ ]:
red_lowres.where(red_lowres != red_lowres.rio.nodata)

In [ ]:
red_nodata.plot(robust=True)

## Raster Bands

In [ ]:
visual_lowres = rioxarray.open_rasterio('../data/input_data/sentinel2/visual.tif', overview_level=2)
visual_lowres

In [ ]:
visual_lowres.shape

In [ ]:
visual_lowres.plot()

In [ ]:
visual_lowres.plot.imshow()